In [1]:
#READ_CSV, READ_TEXT, Import Pandas, os

import pandas as pd
import os

def read_csv(path):
    df = pd.read_csv(path, encoding='unicode_escape')
    df['PersonID'] = os.path.split(path)[1].split('_')[0][6:]
    df['ImageID'] = os.path.split(path)[1].split('.')[0][6:]
    return df

#只有 Text 跟 Caption 會用
def read_text(path, colName):
    df = pd.read_csv(path, sep='\t', encoding='unicode_escape', names=[colName])
    filename = os.path.split(path)[1]
    if 'text' in filename:
        df['TextID'] = filename[6:-4]
    else:
        df['ImageID'] = os.path.split(path)[1].split('.')[0][6:-7]
    return df


In [15]:
#Reading all file

DataPath = 'MC2-Image-Data/'

#Full DataPaths
CsvList = []
#ImageList = [] 
CapList = [] 
TextList = []

#ImagesNames = []

for r, d, files in os.walk(DataPath):

    if 'TrainingImages' in r:
            continue

    for _file in files:

        if _file.endswith('.csv'):
            CsvList.append(r + '/' + _file)
        #elif _file.endswith('.jpg'):
        #    ImageList.append(r + '/' +_file)
        #    ImagesNames.append(_file)
        elif 'caption.txt' in _file:
            CapList.append(r + '/' +_file)
        elif 'text' in _file:
            TextList.append(r + '/' +_file)

In [16]:
# Make Caption/Text/Csv DataFrame

tmp = []
for path in CapList:
    tmp.append(read_text(path, 'Caption'))

cap_df = pd.concat(tmp, ignore_index=True)

tmp = []
for path in TextList:
    tmp.append(read_text(path, 'Text'))

text_df = pd.concat(tmp, ignore_index=True)

tmp = []
for path in CsvList:
    tmp.append(read_csv(path))

csv_df = pd.concat(tmp, ignore_index=True)

In [17]:
# Sort by Simply ImageID
csv_df['Sort'] = csv_df['ImageID'].apply(lambda x : int(x.replace('_', ""))) 
csv_df = csv_df.sort_values(by='Sort', ignore_index=True)
csv_df = csv_df.drop(columns = ['Sort'])
csv_df



,x,y,Width,Height,Score,Label,PersonID,ImageID
0,2554,313,955,888,0.28139,eyeball,1,1_1
1,1855,1724,578,1079,0.34569,pinkCandle,1,1_1
2,14,666,1465,2309,0.42683,pumpkinNotes,1,1_1
3,1560,1301,1197,615,0.32564,blueSunglasses,1,1_1
4,14,464,1775,2268,0.38394,yellowBag,1,1_1
...,...,...,...,...,...,...,...,...
4486,18,1222,2074,2247,0.25895,yellowBag,40,40_10
4487,1279,957,1521,1371,0.48834,hairClip,40,40_10
4488,847,1556,1590,1544,0.58092,redWhistle,40,40_10
4489,1055,795,1953,1602,0.30762,birdCall,40,40_10


In [18]:
# Save Files
csv_df.to_json('FineData.json', orient='records')
csv_df


,x,y,Width,Height,Score,Label,PersonID,ImageID
0,2554,313,955,888,0.28139,eyeball,1,1_1
1,1855,1724,578,1079,0.34569,pinkCandle,1,1_1
2,14,666,1465,2309,0.42683,pumpkinNotes,1,1_1
3,1560,1301,1197,615,0.32564,blueSunglasses,1,1_1
4,14,464,1775,2268,0.38394,yellowBag,1,1_1
...,...,...,...,...,...,...,...,...
4486,18,1222,2074,2247,0.25895,yellowBag,40,40_10
4487,1279,957,1521,1371,0.48834,hairClip,40,40_10
4488,847,1556,1590,1544,0.58092,redWhistle,40,40_10
4489,1055,795,1953,1602,0.30762,birdCall,40,40_10


In [22]:
# Load FineData
Df = pd.read_json('FineData.json', orient='records', dtype=False)
Df


,x,y,Width,Height,Score,Label,PersonID,ImageID
0,2554,313,955,888,0.28139,eyeball,1,1_1
1,1855,1724,578,1079,0.34569,pinkCandle,1,1_1
2,14,666,1465,2309,0.42683,pumpkinNotes,1,1_1
3,1560,1301,1197,615,0.32564,blueSunglasses,1,1_1
4,14,464,1775,2268,0.38394,yellowBag,1,1_1
...,...,...,...,...,...,...,...,...
4486,18,1222,2074,2247,0.25895,yellowBag,40,40_10
4487,1279,957,1521,1371,0.48834,hairClip,40,40_10
4488,847,1556,1590,1544,0.58092,redWhistle,40,40_10
4489,1055,795,1953,1602,0.30762,birdCall,40,40_10


In [23]:
# Find PCA
import numpy as np
from sklearn import preprocessing
from sklearn import decomposition

pivot = Df.pivot(index='ImageID', columns='Label', values='Score').fillna(0.0).astype(np.float)
#print(pivot.shape)

target = pd.DataFrame(pivot.index)
target['PersonID'] = target['ImageID'].apply(lambda x : x.split('_')[0])
#print(target.shape)

X = pivot.to_numpy()
X = preprocessing.scale(X)

pca = decomposition.PCA(n_components=2)
X_r = pca.fit_transform(X)

pca_df = pd.DataFrame(X_r, columns=['PC1', 'PC2'])


# Concat Target and PCA
res = pd.concat([target, pca_df], axis=1)
res

,ImageID,PersonID,PC1,PC2
0,10_1,10,0.254840,-1.394797
1,10_10,10,-1.037851,1.149821
2,10_11,10,-0.919088,1.095798
3,10_12,10,0.309356,-0.378061
4,10_13,10,-1.293908,-1.748523
...,...,...,...,...
889,9_5,9,-1.313600,-1.120599
890,9_6,9,-0.164981,0.763328
891,9_7,9,0.913529,1.366127
892,9_8,9,3.071218,0.647996


In [12]:
#Save PCA fineData
res.to_json('PCA.json', orient='records')

NameError: name 'res' is not defined

In [41]:
#Load PCA
PCA_df = pd.read_json('PCA.json', orient='records', dtype=False)
PCA_df.head()

,ImageID,PersonID,PC1,PC2
0,10_1,10,0.254840,-1.394797
1,10_10,10,-1.037851,1.149821
2,10_11,10,-0.919088,1.095798
3,10_12,10,0.309356,-0.378061
4,10_13,10,-1.293908,-1.748523


In [70]:
#Start Ground Truth
#ref <https://github.com/gj0706/VAST2020_MC2/tree/master/data>

#Load Ground Truth
ground_truth_df = pd.read_json('gt.json', dtype=False)
ground_truth_df.head()

ground_truth_df.columns=['y','x', 'Height', 'Width', 'ImageID', 'PersonID', 'Label']
ground_truth_df

,y,x,Height,Width,ImageID,PersonID,Label
0,427,288,650,672,37_9,37,cloudSign
1,335,2842,2634,730,37_9,37,noisemaker
2,335,3223,2664,763,37_9,37,noisemaker
3,298,1038,888,1000,37_9,37,paperPlate
4,642,2062,488,688,37_9,37,partyFavor
...,...,...,...,...,...,...,...
1353,1386,2776,242,324,34_44,34,turtle
1354,1972,1508,524,362,5_6,5,legoBracelet
1355,1712,1940,232,629,5_6,5,redWhistle
1356,781,2564,330,682,5_6,5,pinkEraser


In [75]:
#Concat Ground Truth and PCA
gt_df = pd.merge(ground_truth_df, PCA_df) #順便移除含null列
gt_df = gt_df.drop(['PC1','PC2'], axis=1)

In [76]:
#Find Ground Truth PCA
import numpy as np
from sklearn import preprocessing
from sklearn import decomposition

final = gt_df
final['Score'] = 1
final = final.drop_duplicates(subset=['PersonID', 'ImageID', 'Label'])
final = final
pivot = pd.pivot(final, index='ImageID', columns='Label', values='Score').fillna(0).astype(np.float)

target = pd.DataFrame(pivot.index)
target['PersonID'] = target['ImageID'].apply(lambda x : x.split('_')[0])


X = pivot.to_numpy()
X = preprocessing.scale(X)

pca = decomposition.PCA(n_components=2)
X_r = pca.fit_transform(X)

pca_df = pd.DataFrame(X_r, columns=['PC1', 'PC2'])
gt_pca = pd.concat([target, pca_df], axis=1)


In [77]:
# Save gt_pca
gt_pca.to_json('gt_pca.json', orient='records')
gt_pca.shape

(836, 4)

In [79]:
gt_pca = pd.read_json('gt_pca.json')
gt_pca

,ImageID,PersonID,PC1,PC2
0,101,10,-2.448205,2.265939
1,1010,10,0.230774,0.022376
2,1011,10,0.230774,0.022376
3,1012,10,-0.004004,-0.881544
4,1013,10,-0.004004,-0.881544
...,...,...,...,...
831,95,9,0.146416,-0.731510
832,96,9,-0.974699,0.626564
833,97,9,-0.974699,0.626564
834,98,9,-1.750895,-0.520572
